# Lesson 3 // Web scraping

---
Web scraping is the process of extracting data from websites. It can be done manually, but typically when we talk of web scraping we mean gathering data from websites by automated means. Web scraping involves two distinct processes: fetching or downloading the web page, and extracting data from it. In this lesson we introduce the **rvest** package, which provides various web scraping functions. 

In this lesson we'll:

1. introduce the SelectorGadget tool and show how to use it to identify the parts of a webpage we want.
2. use **rvest**'s key functions - `read_html()`, `html_nodes()`, and `html_text()` - to scrape data from the web.
3. see how to scrape data tables from the web.
4. use `html_attr()` to get HTML nodes of a particular type, like hyperlinks.
5. use what we've learned to build two larger examples, scraping property data and movie reviews.

Web scraping involves working with HTML files, the language used to construct web pages. The better you know HTML, the easier web scraping will be and the more you can do. That said, this notebook is written as a practical "how to" guide to doing web scraping in R, and tries to get you up and running as quickly as possible. We introduce bits and pieces of HTML as needed, but do not cover these from first principles or in great detail. There is a nice basic introduction to HTML [here](http://www.simplehtmlguide.com/). 

Nevertheless, it will be useful to have a rough idea how everything fits together, which is summarised below:

* Websites are written using **HTML** (Hypertext Markup Language), a markup programming language. A web page is basically an HTML file. An HTML file is a plain-text file in which the text is written using the HTML language i.e. contains HTML commands, content, etc. HTML files can be linked to one another, which is how a web site is put together.

* An HTML file, and hence a web page, consists of two main parts: HTML **tags** and content. HTML tags are the parts of a web page that define how content is formatted and displayed in a web browser. Its easiest to explain with a small example. Below is a minimal HTML file: the tags are the commands within angle brackets e.g. `<head>`. Try copying the text below to a text editor, save as .html, and open in your browser. Tags can be customised with **tag attributes**.

---
```<html>
 <head>
  <title>A simple webpage</title>
 </head>
 <body>

  Some content. More <b>very important</b> content.

 </body>
</html>```
---

* CSS is **Cascading Style Sheets**, a "style sheet language". A style sheet language is a programming language that controls how certain kinds of documents are structured. CSS is a style sheet language for markup documents like those written using HTML. Style sheets define things like the colour and layout of text and other HTML tags. Separating presentation from content is often useful e.g. multiple HTML pages can share formatting through a shared CSS (.css) file.

* A CSS file is written as a set of rules. Each rule consists of a **selector** and a declaration. The CSS selector points to the HTML element the declaration refers to. The declaration contains instructions about how the HTML element identified by the CSS selector should be presented. CSS selectors identify HTML elements by matching tags and tag attributes. There's a fun tutorial on CSS selectors [here](http://flukeout.github.io/).

* **rvest** uses CSS selectors to identify the parts of the web page to scrape.

> Please note! Web scraping invariably involves copying data, and thus copyright issues are often involved. Beyond that, automated web scraping software can process data much more quickly that manual web users, placing a strain on host web servers. Scraping may also be against the terms of service of some websites. The bottom line is that the ethics of web scraping is not straightforward, and is evolving. There is lots of useful information on the web about these issues, for example [here](https://medium.com/towards-data-science/ethics-in-web-scraping-b96b18136f01), [here](http://gijn.org/2015/08/12/on-the-ethics-of-web-scraping-and-data-journalism/), and [here](http://gijn.org/2015/08/12/on-the-ethics-of-web-scraping-and-data-journalism/) (reading up on these is one of the exercises at the end of the notebook). 

---

First we load the packages we'll need in this workbook.

In [1]:
library(rvest)
library(tidyverse)
library(stringr)

Loading required package: xml2
Loading tidyverse: ggplot2
Loading tidyverse: tibble
Loading tidyverse: tidyr
Loading tidyverse: readr
Loading tidyverse: purrr
Loading tidyverse: dplyr
Conflicts with tidy packages ---------------------------------------------------
filter(): dplyr, stats
lag():    dplyr, stats


## Example 1: A simple example to illustrate the use of Selector Gadget

In this example we'll visit the [Eyewitness News](http://ewn.co.za) webpage and use the **SelectorGadget** tool to find the CSS selectors for headlines . Then we'll use the **rvest** package to scrape the headings and save them as strings in R.

First, make sure you've got the SelectorGadget tool available in your web browser's toolbar. Go to http://selectorgadget.com/ and find the link that says "drag this link to your bookmark bar": do that. You only need to do this once. 

Now let's visit the [Eyewitness News](http://ewn.co.za) webpage. Click on the SelectorGadget tool and identify the CSS selectors for headlines (should be `.article-short h4, h1`, although this may change with time). I'll show you how to do this in class, or follow the tutorial on the SelectorGadget website.

Finally, let's switch over to R and scrape the headlines. 

We first read in the webpage using `read_html`. This simply reads in an HTML document, which can be from a url, a file on disk or a string. It returns an XML (another markup language) document.

In [2]:
ewn_page <- read_html("http://ewn.co.za/")
ewn_page

{xml_document}
<html lang="en">
[1] <head>\n<meta http-equiv="Content-Type" content="text/html; charset=UTF-8 ...
[2] <script type="text/javascript">window.NREUM||(NREUM={});NREUM.info = {"be ...
[3] <script type="text/javascript">window.NREUM||(NREUM={}),__nr_require=func ...
[4] <body class="campaign">\r\n    <!-- Google Tag Manager (noscript) -->\r\n ...

We extract relevant information from the document with `html_nodes`. This returns a set of XML element nodes, each one containing the tag and contents (e.g. text) associated with the specified CSS selectors:

In [3]:
ewn_elements <- html_nodes(x = ewn_page, css = ".article-short h4 , h1")
ewn_elements

{xml_nodeset (44)}
 [1] <h4 class="h4-mega text-center"><a href="http://ewn.co.za/Multimedia/Vid ...
 [2] <h4 class="h4-mega text-center"><a href="http://ewn.co.za/Multimedia/Aud ...
 [3] <h4 class="h4-mega text-center"><a href="http://ewn.co.za/Multimedia/Inf ...
 [4] <h4 class="h4-mega text-center"><a href="http://ewn.co.za/Multimedia/Car ...
 [5] <h1>\r\n                                            SABC in hot water ag ...
 [6] <h4>\r\n                                            Sisulu: ANC needs ne ...
 [7] <h4>\r\n                                            Cosatu: All members  ...
 [8] <h4>\r\n                                            Decision to keep Mye ...
 [9] <h4><a href="http://ewn.co.za/2017/09/25/french-unions-block-fuel-depots ...
[10] <h4><a href="http://ewn.co.za/2017/09/25/gunman-kills-woman-wounds-6-at- ...
[11] <h4><a href="http://ewn.co.za/2017/09/25/suspected-drug-dealer-to-appear ...
[12] <h4><a href="http://ewn.co.za/2017/09/25/drivers-using-motor-shows-to-pr .

To get just the text inside the element nodes we use `html_text`, with `trim = TRUE` to clean up whitespace characters.

In [4]:
ewn_text <- html_text(ewn_elements, trim = TRUE) 
as.tibble(ewn_text)

value
Video
Audio
Infographics
Cartoons
SABC in hot water again over Dlamini-Zuma referencing
Sisulu: ANC needs new vision that embraces those who tell the truth
Cosatu: All members behind strike against state capture
Decision to keep Myeni at SAA is irrational - DA
French unions block fuel depots in protest against labour reforms
"Gunman kills woman, wounds 6 at Nashville church"


The table above contains some stuff we don't want (like [WATCH]). We'll look at ways to clean up text later.

## Example 2: Scraping tables

One especially useful form of scaping is getting tables containing data from websites. This example shows you how to do that. 

We'll use the table on [this ESPN cricinfo webpage](http://stats.espncricinfo.com/ci/engine/records/averages/batting.html?class=1;id=2017;type=year), which contains 2017 test cricket batting averages. Before running the code below, visit the webpage and use SelectorGadget to identify the CSS selector you need. Also familiarise yourself with the table, just so you know what to expect.

First, read the webpage as before:

In [5]:
cric_page <- read_html("http://stats.espncricinfo.com/ci/engine/records/averages/batting.html?class=1;id=2017;type=year")

Extract the table element(s) with `html_nodes()`.

In [6]:
cric_elements <- html_nodes(x = cric_page, css = "table")

View the extracted elements, and see we only want the first one.

In [7]:
cric_elements

{xml_nodeset (3)}
[1] <table class="engineTable">\n<caption>Batting averages</caption>\n<thead> ...
[2] <table class="engineTable" id="shading-desc">\n<tr class="data2" style="b ...
[3] <table class="engineTable" style="margin:0px;">\n<tr class="data2">\n<td  ...

Use `html_table()` to extract the tables inside the first element of `cric_elements`.

In [8]:
cric_table <- html_table(cric_elements[[1]])
print(cric_table)

                           Player Mat Inns NO Runs   HS   Ave   BF     SR 100
1                  KJ Abbott (SA)   1    1  0   16   16 16.00   26  61.53   0
2                   AC Agar (AUS)   2    3  1   65  41* 32.50  145  44.82   0
3             Ahmed Shehzad (PAK)   2    4  0  121   70 30.25  310  39.03   0
4                    MM Ali (ENG)   7   12  1  361   87 32.81  496  72.78   0
5                    HM Amla (SA)   9   17  1  645  134 40.31 1327  48.60   1
6               JM Anderson (ENG)   7    9  6   27   12  9.00   60  45.00   0
7               Asad Shafiq (PAK)   4    7  0  101   30 14.42  282  35.81   0
8                R Ashwin (INDIA)   8   10  0  219   54 21.90  375  58.40   0
9                 Azhar Ali (PAK)   4    8  0  343  127 42.87  884  38.80   2
10               Babar Azam (PAK)   4    8  1  145   72 20.71  376  38.56   0
11              JM Bairstow (ENG)   7   12  0  389   99 32.41  607  64.08   0
12              GS Ballance (ENG)   2    4  0   85   34 21.25  1

We can also use the pipe for this. Note the use of `.[[i]]`, which is the operation "extract the *i*-th element".

In [9]:
cric_table_piped <- cric_page %>% html_nodes("table") %>% .[[1]] %>%  html_table() 
print(cric_table_piped)

                           Player Mat Inns NO Runs   HS   Ave   BF     SR 100
1                  KJ Abbott (SA)   1    1  0   16   16 16.00   26  61.53   0
2                   AC Agar (AUS)   2    3  1   65  41* 32.50  145  44.82   0
3             Ahmed Shehzad (PAK)   2    4  0  121   70 30.25  310  39.03   0
4                    MM Ali (ENG)   7   12  1  361   87 32.81  496  72.78   0
5                    HM Amla (SA)   9   17  1  645  134 40.31 1327  48.60   1
6               JM Anderson (ENG)   7    9  6   27   12  9.00   60  45.00   0
7               Asad Shafiq (PAK)   4    7  0  101   30 14.42  282  35.81   0
8                R Ashwin (INDIA)   8   10  0  219   54 21.90  375  58.40   0
9                 Azhar Ali (PAK)   4    8  0  343  127 42.87  884  38.80   2
10               Babar Azam (PAK)   4    8  1  145   72 20.71  376  38.56   0
11              JM Bairstow (ENG)   7   12  0  389   99 32.41  607  64.08   0
12              GS Ballance (ENG)   2    4  0   85   34 21.25  1

# Example 3: Scraping house property data

This is a more advanced example where we scrape data on houses for sale in a particular area of interest. 

The landing page for a suburb shows summaries for the first 20 houses. At the bottom of the page are links to a further pages, each containing 20 house summaries. First we read in the landing page and identify *all* hyperlinks on that page.

In [10]:
suburb <- read_html("https://www.property24.com/for-sale/fish-hoek/cape-town/western-cape/9074")
suburb_links <- suburb %>% html_nodes("a") %>% html_attr("href") 
print(suburb_links)

  [1] "http://windows.microsoft.com/en-US/internet-explorer/download-ie"                                            
  [2] "/"                                                                                                           
  [3] "/"                                                                                                           
  [4] "/for-sale/fish-hoek/fish-hoek/western-cape/9074"                                                             
  [5] "/new-developments/fish-hoek/fish-hoek/western-cape/9074"                                                     
  [6] "/commercial-property-for-sale/fish-hoek/fish-hoek/western-cape/9074"                                         
  [7] "/for-sale/fish-hoek/fish-hoek/western-cape/9074?sp=r%3dTrue"                                                 
  [8] "/for-sale/fish-hoek/fish-hoek/western-cape/9074?sp=oa%3dTrue"                                                
  [9] "/for-sale/fish-hoek/fish-hoek/western-cape/9074?sp=os%3dT

Next, we need to identify just those hyperlinks that load pages with house summaries (I'll call these "summary pages"). We do this by matching pattern with regular expressions.

In [11]:
suburb_pages <- str_subset(suburb_links,"(http).*(for-sale).*(9074)")
suburb_pages

[1] "https://www.property24.com/for-sale/fish-hoek/fish-hoek/western-cape/9074/p2"
[2] "https://www.property24.com/for-sale/fish-hoek/fish-hoek/western-cape/9074"   
[3] "https://www.property24.com/for-sale/fish-hoek/fish-hoek/western-cape/9074/p2"
[4] "https://www.property24.com/for-sale/fish-hoek/fish-hoek/western-cape/9074/p3"
[5] "https://www.property24.com/for-sale/fish-hoek/fish-hoek/western-cape/9074/p4"
[6] "https://www.property24.com/for-sale/fish-hoek/fish-hoek/western-cape/9074/p5"

For each of the summary pages, we extract the hyperlinks that lead to the full house ads

In [12]:
house_links <- c()
for(i in suburb_pages){
  suburb_i <- read_html(i)
  suburb_i_links <- suburb_i %>% html_nodes("a") %>% html_attr("href") 
  house_links_i <- str_subset(suburb_i_links,"(for-sale).*(9074/)[0-9]")
  house_links <- c(house_links, house_links_i)
}
# remove any duplicates and reorder
house_links <- sample(unique(house_links))

In [13]:
house_links

[1] "/for-sale/fish-hoek/fish-hoek/western-cape/9074/104115463"
  [2] "/for-sale/fish-hoek/fish-hoek/western-cape/9074/105580695"
  [3] "/for-sale/fish-hoek/fish-hoek/western-cape/9074/105442251"
  [4] "/for-sale/fish-hoek/fish-hoek/western-cape/9074/105618554"
  [5] "/for-sale/fish-hoek/fish-hoek/western-cape/9074/105609766"
  [6] "/for-sale/fish-hoek/fish-hoek/western-cape/9074/105554320"
  [7] "/for-sale/fish-hoek/fish-hoek/western-cape/9074/105483416"
  [8] "/for-sale/fish-hoek/fish-hoek/western-cape/9074/105436009"
  [9] "/for-sale/fish-hoek/fish-hoek/western-cape/9074/105282092"
 [10] "/for-sale/fish-hoek/fish-hoek/western-cape/9074/104572110"
 [11] "/for-sale/fish-hoek/fish-hoek/western-cape/9074/105583523"
 [12] "/for-sale/fish-hoek/fish-hoek/western-cape/9074/105548402"
 [13] "/for-sale/fish-hoek/fish-hoek/western-cape/9074/105503879"
 [14] "/for-sale/fish-hoek/fish-hoek/western-cape/9074/105436211"
 [15] "/for-sale/fish-hoek/fish-hoek/western-cape/9074/105494708"
 [16] "/for-sale/fish-hoek/fish-hoek/western-cape/9074/105323680"
 [17] "/for-sale/fish-hoek/fish-hoek/western-cape/9074/105311888"
 [18] "/for-sale/fish-hoek/fish-hoek/western-cape/9074/104883083"
 [19] "/for-sale/fish-hoek/fish-hoek/western-cape/9074/105497695"
 [20] "/for-sale/fish-hoek/fish-hoek/western-cape/9074/105440012"
 [21] "/for-sale/fish-hoek/fish-hoek/western-cape/9074/105349318"
 [22] "/for-sale/fish-hoek/fish-hoek/western-cape/9074/105418281"
 [23] "/for-sale/fish-hoek/fish-hoek/western-cape/9074/105392855"
 [24] "/for-sale/fish-hoek/fish-hoek/western-cape/9074/101746997"
 [25] "/for-sale/fish-hoek/fish-hoek/western-cape/9074/105545616"
 [26] "/for-sale/fish-hoek/fish-hoek/western-cape/9074/104931739"
 [27] "/for-sale/fish-hoek/fish-hoek/western-cape/9074/105443836"
 [28] "/for-sale/fish-hoek/fish-hoek/western-cape/9074/105500172"
 [29] "/for-sale/fish-hoek/fish-hoek/western-cape/9074/104788125"
 [30] "/for-sale/fish-hoek/fish-hoek/western-cape/9074/104796703"
 [31] "/for-sale/fish-hoek/fish-hoek/western-cape/9074/104184529"
 [32] "/for-sale/fish-hoek/fish-hoek/western-cape/9074/105590399"
 [33] "/for-sale/fish-hoek/fish-hoek/western-cape/9074/101732759"
 [34] "/for-sale/fish-hoek/fish-hoek/western-cape/9074/105426736"
 [35] "/for-sale/fish-hoek/fish-hoek/western-cape/9074/105584923"
 [36] "/for-sale/fish-hoek/fish-hoek/western-cape/9074/105555928"
 [37] "/for-sale/fish-hoek/fish-hoek/western-cape/9074/105380865"
 [38] "/for-sale/fish-hoek/fish-hoek/western-cape/9074/105506487"
 [39] "/for-sale/fish-hoek/fish-hoek/western-cape/9074/105467451"
 [40] "/for-sale/fish-hoek/fish-hoek/western-cape/9074/105185133"
 [41] "/for-sale/fish-hoek/fish-hoek/western-cape/9074/105455606"
 [42] "/for-sale/fish-hoek/fish-hoek/western-cape/9074/105419355"
 [43] "/for-sale/fish-hoek/fish-hoek/western-cape/9074/105444036"
 [44] "/for-sale/fish-hoek/fish-hoek/western-cape/9074/105190873"
 [45] "/for-sale/fish-hoek/fish-hoek/western-cape/9074/105458932"
 [46] "/for-sale/fish-hoek/fish-hoek/western-cape/9074/101752671"
 [47] "/for-sale/fish-hoek/fish-hoek/western-cape/9074/105027751"
 [48] "/for-sale/fish-hoek/fish-hoek/western-cape/9074/104838272"
 [49] "/for-sale/fish-hoek/fish-hoek/western-cape/9074/105403095"
 [50] "/for-sale/fish-hoek/fish-hoek/western-cape/9074/105562529"
 [51] "/for-sale/fish-hoek/fish-hoek/western-cape/9074/104001630"
 [52] "/for-sale/fish-hoek/fish-hoek/western-cape/9074/105548860"
 [53] "/for-sale/fish-hoek/fish-hoek/western-cape/9074/105437275"
 [54] "/for-sale/fish-hoek/fish-hoek/western-cape/9074/104540282"
 [55] "/for-sale/fish-hoek/fish-hoek/western-cape/9074/105565804"
 [56] "/for-sale/fish-hoek/fish-hoek/western-cape/9074/105209718"
 [57] "/for-sale/fish-hoek/fish-hoek/western-cape/9074/105610509"
 [58] "/for-sale/fish-hoek/fish-hoek/western-cape/9074/105503433"
 [59] "/for-sale/fish-hoek/fish-hoek/western-cape/9074/105421138"
 [60] "/for-sale/fish-hoek/fish-hoek/western-cape/9074/105357913"
 [61] "/for-sale/fish-hoek/fish-hoek/weste

We now read each of those pages and extract the data we want.

In [14]:
house_data <- data.frame()
for(i in house_links[1:3]){   # more than 3 and you get blocked
  
      # read house ad html
  house <- read_html(paste("https://www.property24.com",i, sep=""))
  
      # get the ad text 
  ad <- house %>% html_nodes(css = ".js_readMore") %>% html_text(trim = T)

    # get house data
  price <- house %>% html_nodes(css = ".p24_price") %>% html_text(trim = TRUE)
  erfsize <- house %>% html_nodes(css = ".dropdown-toggle span") 
  nbeds <- house %>% html_nodes(css = ".p24_text:nth-child(2)") %>% html_text(trim = TRUE) %>% as.numeric()
  nbaths <- house %>% html_nodes(css = ".p24_text:nth-child(5)") %>% html_text(trim = TRUE) %>% as.numeric()
  ngar <- house %>% html_nodes(css = ".p24_text:nth-child(8)") %>% html_text(trim = TRUE) %>% as.numeric()

    # if couldn't find data on webpage, replace with NA
  price <- ifelse(length(price) > 0, price, NA)
  erfsize <- ifelse(length(erfsize) > 0, html_text(erfsize, trim = TRUE), NA)
  nbeds <- ifelse(length(nbeds) > 0, nbeds, NA)
  nbaths <- ifelse(length(nbaths) > 0, nbaths, NA)
  ngar <- ifelse(length(ngar) > 0, ngar, NA)
  
    # store results
  this_house <- data.frame(price = price, erfsize = erfsize, nbeds = nbeds, nbaths = nbaths, ngar = ngar, ad = ad)
  house_data <- rbind.data.frame(house_data,this_house)
  
    # random wait avoids excessive requesting
  Sys.sleep(sample(seq(10, 30, by=1), 1))
  
}

View the data

In [15]:
house_data

price,erfsize,nbeds,nbaths,ngar,ad
R 1 200 000,NA,NA,NA,NA,Beautiful plot with the best address and awesome sea views. Watch the wales and dolphins frolic amongst the sail boats below.
R 2 395 000,496 m²,3,2,NA,"Priced to sell this neat Fish Hoek valley home offers more than meets the eye.Featuring 3 Large bedrooms,2 modern bathrooms and a cozy bachelor flat, this home is one of a kind. The crowning jewel being the open living area with kitchen, lounge and dining room all incorporated around a central island. The Kitchen is fitted with dual eye level oven and separate cook top, 2 plumbing points for appliances and illuminated cupboards, all finished in Granite. The airy lounge boasts large windows and a cozy wood burning stove. Dining room leading onto a tiled, covered entertainment area with its own fireplace, just ready for a braai area or lazing on summer days. All Bedrooms with BICs and laminate flooring throughout the home. Tidy well kept garden with well-point makes this home a pleasure to own indeed. It will sell quickly so contact me today if you do not want to miss … Read more"
R 1 100 000,85 m²,2,1,1,Put your finishing touches to this very spacious and secure apartment and you will have a magnificent gem. Close to all amenities as well as the beach. Two spacious bedrooms with ample built in cupboards. The spacious living area is north facing with lovely parquet flooring. 20 sqm single lock up garage. It should be noted that there is a lease in place expiring March 2018


# Example 4: Getting movie reviews

In [16]:
load("data/movielens-small.RData")
load("output/recommender.RData")
# make into a tibble
links <- as.tibble(links)
head(links)

movieId,imdbId,tmdbId
1,114709,862
2,113497,8844
3,113228,15602
4,114885,31357
5,113041,11862
6,113277,949


The *links* data frame provides identifiers for each movie for three different movie datasets: [MovieLens](https://movielens.org), [IMDb](http://www.imdb.com/), and [The Movie Database](https://www.themoviedb.org). This gives us a way of looking up reviews for a particular *movieId* we are interested in on either IMDb or The Movie Database.

IMDb links are 7 characters long, so we need to add leading zeros in some cases.

In [17]:
links$imdbId <- sprintf("%07d",links$imdbId)

Let's extract just the movies that we used to build our recommender systems in the last lesson, and get the IMDB identifiers for those movies.

In [18]:
movies_to_use <- unique(ratings_red$movieId)
imdbId_to_use <- links %>% filter(movieId %in% movies_to_use) 

Next we need to know a little more about how reviews are displayed on IMDb. Firstly, a certain number of reviews are shown per page, as in the property example, so we need a way to handle that. Secondly, we need to know the CSS selector for the review text we want to scrape.

In [19]:
reviews <- data.frame()

# just get the first two movies to save time
for(j in 1:2){

  this_movie <- imdbId_to_use$imdbId[j]

    # just get the first 50 reviews
  for(i in c(0, seq(10, 50, 10))) {
    link <- paste0("http://www.imdb.com/title/tt",this_movie,"/reviews?start=",i)
    movie_imdb <- read_html(link)
    
    # Used SelectorGadget as the CSS Selector
    imdb_review <- movie_imdb %>% html_nodes("#pagecontent") %>%
      html_nodes("div+p") %>%
      html_text()
        
    this_review <- data.frame(imbdId = this_movie, review = imdb_review)
    reviews <- rbind.data.frame(reviews, this_review)
  }
  
}

reviews <- as.tibble(reviews)

We'll now look in a bit more detail on working with text. Let's look at the first review.

In [20]:
review1 <- as.character(reviews$review[1])
review1

[1] "\n\nA towering classic of American cinematic power.  Martin Scorsese teams up\nwith one of the most intense actors of that time to create a masterpiece of\nurban alienation.  Paul Schrader's magnificent script paints a portrait of\nloneliness in the largest city of the world.  Travis never once enters into\na meaningful relationship with any character anywhere in the film.  He is\nthe most hopelessly alone person I've ever encountered on film.He is alone with his thoughts, and his thoughts are dark ones.  The film\nfools you on a first viewing.  Is Travis an endearing eccentric?  Sure, he's\nodd, but he's so polite, and he's got a quirky sense of humor.  His\naffection for Betsy is actually rather endearing.  But on a second view, you\nsee it for what it is.  The audience comes to see Travis's psychosis\ngradually, but there's actually far less development than one might think.\nWhen he talks about cleaning up the city, the repeat viewer knows he doesn't\nmean some sort of Giuliani-facelift.  This is less a film about a character\nin development as it is a kind of snapshot.  To be sure, it takes the\nstimulus to provoke the response, but does that imply some kind of central\nchange in the character?Tremendous supporting roles are brought to life through vivid performances\nby Keitel and Foster especially.  Shepard's character, Betsy, is little more\nthan a foil to highlight Travis's utter alienation from society, but she is\nstill impeccably portrayed.  With only two scenes that don't center on\nTravis, it is unavoidably De Niro's show.  The life with which the\nsupporting cast imbues their characters is a credit to themselves, and to\nthe director's willingness to let the film develop from the intersection of\ndiverse ideas and approaches.  What would the plot lose by eliminating the\nAlbert Brooks character (Tom)?  Nothing at all.  He makes almost no impact\non Travis's life, which is where the plot lives.  But his inclusion makes\nthe film as a whole much richer and fuller.As a piece of American cinema history, this film will live forever.  But far\nmore important than that, this film will survive as a universal,\never-relevant examination of the workings of the alienated mind.  The story\ndoesn't end when the credits roll.  We know Travis will snap again.  But the\nstory doesn't end with Travis either.  It continues today in the cities and\nin the schools.  The film is about the brutal power of the disaffected mind.This film didn't cause the incidents in Colombine, or Hawaii, or Seattle, or\nwherever you care to look, even with all of its disturbing images of\nviolence.  It didn't cause those things.  It predicted them.\n\n"

The first thing we can do is remove references to `\r` and `\n`, which indicate carriage returns and new lines respectively. We do this with a call to `str_replace_all()` and a "regular expression", a way of describing patterns in strings. We'll look at regular expressions in more detail in the next workbook.

In [21]:
review1_nospace <- str_replace_all(review1, "[\r\n]", " ")
review1_nospace

[1] "  A towering classic of American cinematic power.  Martin Scorsese teams up with one of the most intense actors of that time to create a masterpiece of urban alienation.  Paul Schrader's magnificent script paints a portrait of loneliness in the largest city of the world.  Travis never once enters into a meaningful relationship with any character anywhere in the film.  He is the most hopelessly alone person I've ever encountered on film.He is alone with his thoughts, and his thoughts are dark ones.  The film fools you on a first viewing.  Is Travis an endearing eccentric?  Sure, he's odd, but he's so polite, and he's got a quirky sense of humor.  His affection for Betsy is actually rather endearing.  But on a second view, you see it for what it is.  The audience comes to see Travis's psychosis gradually, but there's actually far less development than one might think. When he talks about cleaning up the city, the repeat viewer knows he doesn't mean some sort of Giuliani-facelift.  This is less a film about a character in development as it is a kind of snapshot.  To be sure, it takes the stimulus to provoke the response, but does that imply some kind of central change in the character?Tremendous supporting roles are brought to life through vivid performances by Keitel and Foster especially.  Shepard's character, Betsy, is little more than a foil to highlight Travis's utter alienation from society, but she is still impeccably portrayed.  With only two scenes that don't center on Travis, it is unavoidably De Niro's show.  The life with which the supporting cast imbues their characters is a credit to themselves, and to the director's willingness to let the film develop from the intersection of diverse ideas and approaches.  What would the plot lose by eliminating the Albert Brooks character (Tom)?  Nothing at all.  He makes almost no impact on Travis's life, which is where the plot lives.  But his inclusion makes the film as a whole much richer and fuller.As a piece of American cinema history, this film will live forever.  But far more important than that, this film will survive as a universal, ever-relevant examination of the workings of the alienated mind.  The story doesn't end when the credits roll.  We know Travis will snap again.  But the story doesn't end with Travis either.  It continues today in the cities and in the schools.  The film is about the brutal power of the disaffected mind.This film didn't cause the incidents in Colombine, or Hawaii, or Seattle, or wherever you care to look, even with all of its disturbing images of violence.  It didn't cause those things.  It predicted them.  "

We can remove punctuation in a very similar way. Here `:alnum:` refers to any alphanumeric character, equivalent to `[A-Za-z0-9]`. In this context `^` means negation, so we're removing anything that's not alphanumeric (replacing it with nothing). 

In [22]:
review1_nopunc <- str_replace_all(review1_nospace, "[^[:alnum:] ]", "")
review1_nopunc

[1] "  A towering classic of American cinematic power  Martin Scorsese teams up with one of the most intense actors of that time to create a masterpiece of urban alienation  Paul Schraders magnificent script paints a portrait of loneliness in the largest city of the world  Travis never once enters into a meaningful relationship with any character anywhere in the film  He is the most hopelessly alone person Ive ever encountered on filmHe is alone with his thoughts and his thoughts are dark ones  The film fools you on a first viewing  Is Travis an endearing eccentric  Sure hes odd but hes so polite and hes got a quirky sense of humor  His affection for Betsy is actually rather endearing  But on a second view you see it for what it is  The audience comes to see Traviss psychosis gradually but theres actually far less development than one might think When he talks about cleaning up the city the repeat viewer knows he doesnt mean some sort of Giulianifacelift  This is less a film about a character in development as it is a kind of snapshot  To be sure it takes the stimulus to provoke the response but does that imply some kind of central change in the characterTremendous supporting roles are brought to life through vivid performances by Keitel and Foster especially  Shepards character Betsy is little more than a foil to highlight Traviss utter alienation from society but she is still impeccably portrayed  With only two scenes that dont center on Travis it is unavoidably De Niros show  The life with which the supporting cast imbues their characters is a credit to themselves and to the directors willingness to let the film develop from the intersection of diverse ideas and approaches  What would the plot lose by eliminating the Albert Brooks character Tom  Nothing at all  He makes almost no impact on Traviss life which is where the plot lives  But his inclusion makes the film as a whole much richer and fullerAs a piece of American cinema history this film will live forever  But far more important than that this film will survive as a universal everrelevant examination of the workings of the alienated mind  The story doesnt end when the credits roll  We know Travis will snap again  But the story doesnt end with Travis either  It continues today in the cities and in the schools  The film is about the brutal power of the disaffected mindThis film didnt cause the incidents in Colombine or Hawaii or Seattle or wherever you care to look even with all of its disturbing images of violence  It didnt cause those things  It predicted them  "

Finally we can convert everything to lowercase. Note that there are still some problems we'd like to fix up, most often when two words get concatenated (e.g. "charactertremendous" about half-way through the review). Getting text totally clean can be hard work.

In [23]:
review1_clean <- tolower(review1_nopunc)
review1_clean

[1] "  a towering classic of american cinematic power  martin scorsese teams up with one of the most intense actors of that time to create a masterpiece of urban alienation  paul schraders magnificent script paints a portrait of loneliness in the largest city of the world  travis never once enters into a meaningful relationship with any character anywhere in the film  he is the most hopelessly alone person ive ever encountered on filmhe is alone with his thoughts and his thoughts are dark ones  the film fools you on a first viewing  is travis an endearing eccentric  sure hes odd but hes so polite and hes got a quirky sense of humor  his affection for betsy is actually rather endearing  but on a second view you see it for what it is  the audience comes to see traviss psychosis gradually but theres actually far less development than one might think when he talks about cleaning up the city the repeat viewer knows he doesnt mean some sort of giulianifacelift  this is less a film about a character in development as it is a kind of snapshot  to be sure it takes the stimulus to provoke the response but does that imply some kind of central change in the charactertremendous supporting roles are brought to life through vivid performances by keitel and foster especially  shepards character betsy is little more than a foil to highlight traviss utter alienation from society but she is still impeccably portrayed  with only two scenes that dont center on travis it is unavoidably de niros show  the life with which the supporting cast imbues their characters is a credit to themselves and to the directors willingness to let the film develop from the intersection of diverse ideas and approaches  what would the plot lose by eliminating the albert brooks character tom  nothing at all  he makes almost no impact on traviss life which is where the plot lives  but his inclusion makes the film as a whole much richer and fulleras a piece of american cinema history this film will live forever  but far more important than that this film will survive as a universal everrelevant examination of the workings of the alienated mind  the story doesnt end when the credits roll  we know travis will snap again  but the story doesnt end with travis either  it continues today in the cities and in the schools  the film is about the brutal power of the disaffected mindthis film didnt cause the incidents in colombine or hawaii or seattle or wherever you care to look even with all of its disturbing images of violence  it didnt cause those things  it predicted them  "

## Exercises

> Please note I haven't tried these myself yet, so I am not certain that the exercises will "work". If you run into problems let me know!

1. The [Freakonomics Radio Archive](http://freakonomics.com/archive/) contains all previous Freakonomics podcasts. Scrape the titles, dates and descriptions, and download URLs of all the podcasts and store them in a dataframe (see if you can download all the medically-themed podcasts).

2. [Decanter](http://www.decanter.com/) magazine provides one of the world's best known wine ratings. Scrape the tasting notes, scores, and prices for their South African white wines (or whatever subset you choose). 

3. Think of your own scraping example - a website you think contains useful or interesting information - and put together your own tutorial like one of those above.

4. Web scraping does bring with it some ethical concerns. Its important to read about these and formulate your own opinion and approach, starting for example [here](https://medium.com/towards-data-science/ethics-in-web-scraping-b96b18136f01), [here](http://gijn.org/2015/08/12/on-the-ethics-of-web-scraping-and-data-journalism/), and [here](http://gijn.org/2015/08/12/on-the-ethics-of-web-scraping-and-data-journalism/).